# Recommender with splitted user 

In [79]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')

dataset = pd.read_csv('data_train.csv', names=["user", "item", "interaction"], header=0)

# Get for each user only the number of interaction, this will allow to create group of users 
# based on how much interaction they had
grouped_users = dataset.groupby(['user']).count()
splitted_user = list()

for group_id in range(1, 11):
    grouped_users_local = grouped_users.copy()
    grouped_users_local = grouped_users_local[grouped_users_local.item == group_id]
    grouped_users_local = grouped_users_local.reset_index()['user']
    splitted_user.append(grouped_users_local)
    print("**********: ", group_id, " Lenght: ", len(grouped_users_local))

grouped_users_local = grouped_users.copy()
grouped_users_local = grouped_users_local[grouped_users_local.item > 10]
grouped_users_local = grouped_users_local.reset_index()['user']
splitted_user.append(grouped_users_local)
#print("**********: ", 11, "> Lenght: ", len(grouped_users_local))


**********:  1  Lenght:  93
**********:  2  Lenght:  995
**********:  3  Lenght:  1148
**********:  4  Lenght:  933
**********:  5  Lenght:  605
**********:  6  Lenght:  490
**********:  7  Lenght:  413
**********:  8  Lenght:  607
**********:  9  Lenght:  250
**********:  10  Lenght:  179


# Data pre-processing

In [80]:
users = dataset.user
items = dataset.item
data = dataset.interaction
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [82]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

users_not_in_group = grouped_users.copy()
users_not_in_group = users_not_in_group[users_not_in_group.item != 3]
users_not_in_group = users_not_in_group.reset_index()['user']
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users = users_not_in_group)

print(users_not_in_group)

EvaluatorHoldout: Ignoring 6799 Users
0          0
1          1
2          2
3          3
4          4
        ... 
6794    7941
6795    7942
6796    7944
6797    7945
6798    7946
Name: user, Length: 6799, dtype: int64


In [ ]:
from GraphBased.P3alphaRecommender import P3alphaRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

recommender_alpha = P3alphaRecommender(URM_train)
recommender_alpha.fit(topK=475, alpha = 0.45, implicit = True)

recommender_alpha_ICM = P3alphaRecommender(ICM_all.T)
recommender_alpha_ICM.fit(topK=175, alpha = 0.45)
recommender_alpha_ICM.URM_train = URM_train

hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_train, recommender_alpha_ICM.W_sparse, recommender_alpha.W_sparse)
hybridrecommender.fit(topK=600, alpha = 0.45)

In [ ]:
evaluator_validation.evaluateRecommender(hybridrecommender)